In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
from bs4 import BeautifulSoup
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('stopwords')
nltk.download('punkt_tab')
nltk.download('wordnet')
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\asus\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\asus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\asus\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
df=pd.read_csv('dataset.csv')
df.head()

,id,qid1,qid2,question1,question2,is_duplicate,freq_qid1,freq_qid2,q1len,q2len,q1_n_words,q2_n_words,word_Common,word_Total,word_share,freq_q1+q2,freq_q1-q2
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,1,1,66,57,14,12,10.0,23.0,0.434783,2,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,4,1,51,88,8,13,4.0,20.0,0.200000,5,3
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,1,1,73,59,14,10,4.0,24.0,0.166667,2,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,1,1,50,65,11,9,0.0,19.0,0.000000,2,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,3,1,76,39,13,7,2.0,20.0,0.100000,4,2


In [3]:
df1 = df.sample(80000,random_state=2)
df1.drop([ 'freq_qid1', 'freq_qid2', 'q1len', 'q2len', 'q1_n_words', 'q2_n_words',
        'word_Common', 'word_Total', 'word_share', 'freq_q1+q2', 'freq_q1-q2','qid1','qid2','id'],axis=1,inplace=True)
df1=df1.reset_index(drop=True)

In [4]:
def text_cleaning(q):
  q=str(q).lower().strip()
  ##Replacing certain characters:
  q=q.replace('$','dollar')
  q=q.replace('%','percentage')
  q=q.replace('#','')
  q=q.replace('₹','rupee')
  q=q.replace('€','euro')
  q=q.replace('@','at')

  ##Replacing '[math]' as it convey no meaning.
  q=q.replace('[math]','')

  ##Replacing numbers with their string equivalents:
  q=q.replace(',000,000,000','B')
  q=q.replace(',000,000','M')
  q=q.replace(',000','K')
  q=re.sub(r'([0-9]+)000000000',r'\1b',q)
  q=re.sub(r'([0-9]+)000000',r'\1m',q)
  q=re.sub(r'([0-9]+)000',r'\1k',q)


  ##Decontracting words:
  contractions = {
    "ain't": "am not",
    "aren't": "are not",
    "can't": "can not",
    "can't've": "can not have",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he would",
    "he'd've": "he would have",
    "he'll": "he will",
    "he'll've": "he will have",
    "he's": "he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how is",
    "i'd": "i would",
    "i'd've": "i would have",
    "i'll": "i will",
    "i'll've": "i will have",
    "i'm": "i am",
    "i've": "i have",
    "isn't": "is not",
    "it'd": "it would",
    "it'd've": "it would have",
    "it'll": "it will",
    "it'll've": "it will have",
    "it's": "it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she would",
    "she'd've": "she would have",
    "she'll": "she will",
    "she'll've": "she will have",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so as",
    "that'd": "that would",
    "that'd've": "that would have",
    "that's": "that is",
    "there'd": "there would",
    "there'd've": "there would have",
    "there's": "there is",
    "they'd": "they would",
    "they'd've": "they would have",
    "they'll": "they will",
    "they'll've": "they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we would",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what will",
    "what'll've": "what will have",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "when's": "when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where is",
    "where've": "where have",
    "who'll": "who will",
    "who'll've": "who will have",
    "who's": "who is",
    "who've": "who have",
    "why's": "why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you would",
    "you'd've": "you would have",
    "you'll": "you will",
    "you'll've": "you will have",
    "you're": "you are",
    "you've": "you have"
    }

  q_new=[]
  for word in q.split():
    if word in contractions.keys():
      word=contractions[word]
    q_new.append(word)
  q=' '.join(q_new)
  q = q.replace("'ve", " have")
  q = q.replace("n't", " not")
  q = q.replace("'re", " are")
  q = q.replace("'ll", " will")

  # Removing HTML tags
  q = BeautifulSoup(q)
  q = q.get_text()

  # Remove punctuations
  pattern = re.compile('\W')
  q = re.sub(pattern, ' ', q).strip() ## Characters matched by \W include spaces, punctuation marks, and special symbols like !, @, #, %, etc.


  return q


In [5]:
text="tushar can't be the strongest87, and & intelligent too*"
text_cleaning(text)

'tushar can not be the strongest87  and   intelligent too'

In [6]:
df1['question1']=df1['question1'].apply(text_cleaning)
df1['question2']=df1['question2'].apply(text_cleaning)

df1.head(10)

,question1,question2,is_duplicate
0,what is the best marketing automation tool for...,what is the best marketing automation tool for...,1
1,i am poor but i want to invest what should i do,i am quite poor and i want to be very rich wh...,0
2,i am from india and live abroad i met a guy f...,t i e t to thapar university to thapar univers...,0
3,why do so many people in the u s hate the sou...,my boyfriend doesnt feel guilty when he hurts ...,0
4,consequences of bhopal gas tragedy,what was the reason behind the bhopal gas tragedy,0
5,i killed a snake on a friday there is a belie...,do snakes really take revenge,0
6,is the royal family a net gain or a net loss t...,why does the british royal family think it is ...,0
7,if a huge asteroid was about to hit earth in x...,a 100 mile wide asteroid is projected to hit e...,0
8,what would happen if a woman took viagra,does viagra really work,0
9,how could i improve my love to my girlfriend,how do i improve my relationship with my girlf...,0


In [7]:
df1

,question1,question2,is_duplicate
0,what is the best marketing automation tool for...,what is the best marketing automation tool for...,1
1,i am poor but i want to invest what should i do,i am quite poor and i want to be very rich wh...,0
2,i am from india and live abroad i met a guy f...,t i e t to thapar university to thapar univers...,0
3,why do so many people in the u s hate the sou...,my boyfriend doesnt feel guilty when he hurts ...,0
4,consequences of bhopal gas tragedy,what was the reason behind the bhopal gas tragedy,0
...,...,...,...
79995,what are some tips on making it through the jo...,what are some tips on making it through the jo...,0
79996,which are the best jeans bands in india,what is the best indonesian band,0
79997,what was world like in 1880 s,how was earth s climate before 1880,0
79998,why do people want to become rich in terms of...,why does people want to become rich,1


In [8]:
def fetch_length(q):
  return len(q.split())

In [9]:
df1['q1_length']=df1['question1'].apply(fetch_length)
df1['q2_length']=df1['question2'].apply(fetch_length)

In [10]:
df1

,question1,question2,is_duplicate,q1_length,q2_length
0,what is the best marketing automation tool for...,what is the best marketing automation tool for...,1,13,13
1,i am poor but i want to invest what should i do,i am quite poor and i want to be very rich wh...,0,12,15
2,i am from india and live abroad i met a guy f...,t i e t to thapar university to thapar univers...,0,26,20
3,why do so many people in the u s hate the sou...,my boyfriend doesnt feel guilty when he hurts ...,0,13,30
4,consequences of bhopal gas tragedy,what was the reason behind the bhopal gas tragedy,0,5,9
...,...,...,...,...,...
79995,what are some tips on making it through the jo...,what are some tips on making it through the jo...,0,16,16
79996,which are the best jeans bands in india,what is the best indonesian band,0,8,6
79997,what was world like in 1880 s,how was earth s climate before 1880,0,7,7
79998,why do people want to become rich in terms of...,why does people want to become rich,1,13,7


In [11]:
df1['q1_chars']=df1['question1'].str.len()
df1['q2_chars']=df1['question2'].str.len()
df1

,question1,question2,is_duplicate,q1_length,q2_length,q1_chars,q2_chars
0,what is the best marketing automation tool for...,what is the best marketing automation tool for...,1,13,13,75,76
1,i am poor but i want to invest what should i do,i am quite poor and i want to be very rich wh...,0,12,15,48,56
2,i am from india and live abroad i met a guy f...,t i e t to thapar university to thapar univers...,0,26,20,104,119
3,why do so many people in the u s hate the sou...,my boyfriend doesnt feel guilty when he hurts ...,0,13,30,58,145
4,consequences of bhopal gas tragedy,what was the reason behind the bhopal gas tragedy,0,5,9,34,49
...,...,...,...,...,...,...,...
79995,what are some tips on making it through the jo...,what are some tips on making it through the jo...,0,16,16,94,94
79996,which are the best jeans bands in india,what is the best indonesian band,0,8,6,39,32
79997,what was world like in 1880 s,how was earth s climate before 1880,0,7,7,29,35
79998,why do people want to become rich in terms of...,why does people want to become rich,1,13,7,67,35


In [12]:
def common_words(row):
  # w1=map(lambda word:word.lower().strip(),row['question1'].split(" "))
  # w2=map(lambda word:word.lower().strip(),row['question2'].split(" "))
  list1=row['question1'].split(" ")
  list2=row['question2'].split(" ")
  list1_new=[]
  list2_new=[]
  for word in list1:
    word=word.lower().strip()
    list1_new.append(word)
  set1=set(list1_new)

  for word in list2:
    word=word.lower().strip()
    list2_new.append(word)
  set2=set(list2_new)

  return len(set1 & set2)

In [13]:
df1['word_common']=df1.apply(common_words,axis=1)
df1.tail(5)

,question1,question2,is_duplicate,q1_length,q2_length,q1_chars,q2_chars,word_common
79995,what are some tips on making it through the jo...,what are some tips on making it through the jo...,0,16,16,94,94,15
79996,which are the best jeans bands in india,what is the best indonesian band,0,8,6,39,32,2
79997,what was world like in 1880 s,how was earth s climate before 1880,0,7,7,29,35,3
79998,why do people want to become rich in terms of...,why does people want to become rich,1,13,7,67,35,6
79999,why do so many people ask questions on quora i...,why do people ask question on quora that can b...,1,16,16,90,92,6


In [14]:
df1['total_words']=df1['q1_length']+df1['q2_length']
df1.head(2)

,question1,question2,is_duplicate,q1_length,q2_length,q1_chars,q2_chars,word_common,total_words
0,what is the best marketing automation tool for...,what is the best marketing automation tool for...,1,13,13,75,76,12,26
1,i am poor but i want to invest what should i do,i am quite poor and i want to be very rich wh...,0,12,15,48,56,8,27


In [15]:
df1['word_share'] = round(df1['word_common']/df1['total_words'],2)
df1.head()

,question1,question2,is_duplicate,q1_length,q2_length,q1_chars,q2_chars,word_common,total_words,word_share
0,what is the best marketing automation tool for...,what is the best marketing automation tool for...,1,13,13,75,76,12,26,0.46
1,i am poor but i want to invest what should i do,i am quite poor and i want to be very rich wh...,0,12,15,48,56,8,27,0.30
2,i am from india and live abroad i met a guy f...,t i e t to thapar university to thapar univers...,0,26,20,104,119,4,46,0.09
3,why do so many people in the u s hate the sou...,my boyfriend doesnt feel guilty when he hurts ...,0,13,30,58,145,1,43,0.02
4,consequences of bhopal gas tragedy,what was the reason behind the bhopal gas tragedy,0,5,9,34,49,3,14,0.21


In [16]:
def remove_stopwords(q):
  stop_words = set(stopwords.words('english'))
  lemma = WordNetLemmatizer()

  if pd.isnull(q):  # Handle NaN values
    return ''
  # Tokenize the text
  words = word_tokenize(q)

  # Remove stopwords and lemmatize
  words = [lemma.lemmatize(word) for word in words if word.lower() not in stop_words]

  # Join the words back into a single string
  return ' '.join(words)

In [17]:
df1['question1']=df1['question1'].apply(remove_stopwords)
df1['question2']=df1['question2'].apply(remove_stopwords)

df1.head(10)

,question1,question2,is_duplicate,q1_length,q2_length,q1_chars,q2_chars,word_common,total_words,word_share
0,best marketing automation tool small mid size ...,best marketing automation tool small mid sized...,1,13,13,75,76,12,26,0.46
1,poor want invest,quite poor want rich,0,12,15,48,56,8,27,0.30
2,india live abroad met guy france party want date,e thapar university thapar university institut...,0,26,20,104,119,4,46,0.09
3,many people u hate southern state,boyfriend doesnt feel guilty hurt cried tellin...,0,13,30,58,145,1,43,0.02
4,consequence bhopal gas tragedy,reason behind bhopal gas tragedy,0,5,9,34,49,3,14,0.21
5,killed snake friday belief kill snake friday c...,snake really take revenge,0,29,5,135,29,2,34,0.06
6,royal family net gain net loss british taxpayer,british royal family think ok take taxpayer mo...,0,15,43,68,222,6,58,0.10
7,huge asteroid hit earth x year would able find...,100 mile wide asteroid projected hit earth 50 ...,0,23,17,108,90,8,40,0.20
8,would happen woman took viagra,viagra really work,0,8,4,40,23,1,12,0.08
9,could improve love girlfriend,improve relationship girlfriend,0,9,9,44,51,5,18,0.28


In [18]:
from gensim.models import Word2Vec,KeyedVectors
import gensim.downloader as api

# wv = api.load('word2vec-google-news-300')

In [19]:
# import pandas as pd

# # Assuming 'question1_vector' and 'question2_vector' are numpy arrays in the DataFrame

# # Step 1: Split question1_vector into separate columns
# q1_vector_columns = pd.DataFrame(df['question1_vector'].tolist(),
#                                  columns=[f'q1_vec_{i}' for i in range(len(df['question1_vector'][0]))])

# # Step 2: Split question2_vector into separate columns
# q2_vector_columns = pd.DataFrame(df['question2_vector'].tolist(),
#                                  columns=[f'q2_vec_{i}' for i in range(len(df['question2_vector'][0]))])

# # Step 3: Concatenate the new columns back to the original DataFrame
# df = pd.concat([df, q1_vector_columns, q2_vector_columns], axis=1)

# # Step 4: Drop the original vector columns (optional)
# df = df.drop(['question1_vector', 'question2_vector'], axis=1)

# # Display the updated DataFrame
# df.head(10)


In [20]:
df1.question2.isnull().value_counts()

False    80000
Name: question2, dtype: int64

In [21]:
# sentence = "Tushar is good boy"
# model = Word2Vec([word_tokenize(sentence)], vector_size=100, window=5, min_count=1)
# def sentence_to_vectors(q):

#   sentences=word_tokenize(str(q))
#   word_vectors = [model.wv[word] for word in sentences if word in model.wv]
#   if not word_vectors:
#       return np.zeros(100)
#   return np.mean(word_vectors, axis=0)



In [22]:
# df1['q1_vectors']=df1['question1'].apply(sentence_to_vectors)

In [23]:
df1

,question1,question2,is_duplicate,q1_length,q2_length,q1_chars,q2_chars,word_common,total_words,word_share
0,best marketing automation tool small mid size ...,best marketing automation tool small mid sized...,1,13,13,75,76,12,26,0.46
1,poor want invest,quite poor want rich,0,12,15,48,56,8,27,0.30
2,india live abroad met guy france party want date,e thapar university thapar university institut...,0,26,20,104,119,4,46,0.09
3,many people u hate southern state,boyfriend doesnt feel guilty hurt cried tellin...,0,13,30,58,145,1,43,0.02
4,consequence bhopal gas tragedy,reason behind bhopal gas tragedy,0,5,9,34,49,3,14,0.21
...,...,...,...,...,...,...,...,...,...,...
79995,tip making job interview process strategic hot...,tip making job interview process starwood hote...,0,16,16,94,94,15,32,0.47
79996,best jean band india,best indonesian band,0,8,6,39,32,2,14,0.14
79997,world like 1880,earth climate 1880,0,7,7,29,35,3,14,0.21
79998,people want become rich term money business,people want become rich,1,13,7,67,35,6,20,0.30


In [24]:
df1.question2.isnull().value_counts()

False    80000
Name: question2, dtype: int64

In [25]:
# Function to convert sentences to vectors
def sentence_to_vector(sentence, model, vector_size):

    # Tokenize the sentence
    words = word_tokenize(sentence)

    # Get the Word2Vec vector for each word (ignore words not in the model)
    word_vectors = [model.wv[word] for word in words if word in model.wv]

    # If no valid words are found, return a zero vector
    if not word_vectors:
        return np.zeros(vector_size)

    # Compute the average of the word vectors
    return np.mean(word_vectors, axis=0)


In [26]:
## Step 1: Prepare your data for Word2Vec
sentences_1 = [word_tokenize(str(sentence)) for sentence in df['question1']]
sentences_2 = [word_tokenize(str(sentence)) for sentence in df['question2']]
all_sentences=sentences_1+sentences_2

# Step 2: Train a Word2Vec model on your dataset
vector_size = 100  # Set the dimension of the vectors
word2vec_model = Word2Vec(all_sentences, vector_size=vector_size, window=5, min_count=1)

# Step 3: Convert each sentence to a vector
df1['question1_vector'] = df1['question1'].dropna().apply(lambda x: sentence_to_vector(x, word2vec_model, vector_size))
df1['question2_vector'] = df1['question2'].dropna().apply(lambda x: sentence_to_vector(x, word2vec_model, vector_size))


In [27]:
df1.question1_vector[0].shape

(100,)

In [28]:
df1

,question1,question2,is_duplicate,q1_length,q2_length,q1_chars,q2_chars,word_common,total_words,word_share,question1_vector,question2_vector
0,best marketing automation tool small mid size ...,best marketing automation tool small mid sized...,1,13,13,75,76,12,26,0.46,"[0.13103828, 0.30373496, -0.55339754, 1.544557...","[-4.142523e-06, 0.32123822, -0.46563542, 1.451..."
1,poor want invest,quite poor want rich,0,12,15,48,56,8,27,0.30,"[-0.13162537, -1.531969, -0.46786356, 0.068490...","[0.5783453, -0.94953406, -0.23110044, -0.65063..."
2,india live abroad met guy france party want date,e thapar university thapar university institut...,0,26,20,104,119,4,46,0.09,"[-0.50059694, -0.5758789, -0.8060021, 0.292189...","[-1.0019118, 0.48428267, 0.008479967, 0.356270..."
3,many people u hate southern state,boyfriend doesnt feel guilty hurt cried tellin...,0,13,30,58,145,1,43,0.02,"[-0.72278374, 0.66657054, -0.26804718, -1.0973...","[-0.51271474, 0.36681774, -0.306096, -0.550501..."
4,consequence bhopal gas tragedy,reason behind bhopal gas tragedy,0,5,9,34,49,3,14,0.21,"[-0.29237998, 0.08231849, 0.031954467, -0.0165...","[0.10423759, 0.5441323, -0.5572579, 0.4091199,..."
...,...,...,...,...,...,...,...,...,...,...,...,...
79995,tip making job interview process strategic hot...,tip making job interview process starwood hote...,0,16,16,94,94,15,32,0.47,"[-0.66971964, -0.0012797862, 0.03266107, 1.430...","[-0.7413123, -0.032558143, 0.05396518, 1.63073..."
79996,best jean band india,best indonesian band,0,8,6,39,32,2,14,0.14,"[1.1195953, -0.47903186, -1.2614517, 0.9395758...","[1.7638, 0.00920853, -0.79921573, 0.8824299, 0..."
79997,world like 1880,earth climate 1880,0,7,7,29,35,3,14,0.21,"[0.8219142, -0.662965, -1.4962082, 0.6794536, ...","[0.1587931, 0.51231617, -0.5124019, -0.611486,..."
79998,people want become rich term money business,people want become rich,1,13,7,67,35,6,20,0.30,"[-0.42722934, 0.048682094, 0.047600083, -0.194...","[-0.7485309, -0.1358214, 0.6773087, -0.7571964..."


In [29]:
q1_vector_columns = pd.DataFrame(df1['question1_vector'].tolist(),
                                 columns=[f'q1_vec_{i}' for i in range(len(df1['question1_vector'][0]))])

# Step 2: Split question2_vector into separate columns
q2_vector_columns = pd.DataFrame(df1['question2_vector'].tolist(),
                                 columns=[f'q2_vec_{i}' for i in range(len(df1['question2_vector'][0]))])

# Step 3: Concatenate the new columns back to the original DataFrame
df1 = pd.concat([df1, q1_vector_columns, q2_vector_columns], axis=1)

In [30]:
df1

,question1,question2,is_duplicate,q1_length,q2_length,q1_chars,q2_chars,word_common,total_words,word_share,...,q2_vec_90,q2_vec_91,q2_vec_92,q2_vec_93,q2_vec_94,q2_vec_95,q2_vec_96,q2_vec_97,q2_vec_98,q2_vec_99
0,best marketing automation tool small mid size ...,best marketing automation tool small mid sized...,1,13,13,75,76,12,26,0.46,...,-0.252810,-0.290889,0.145225,0.305381,-0.409946,0.176388,-0.099588,-0.415748,-1.360411,-0.054905
1,poor want invest,quite poor want rich,0,12,15,48,56,8,27,0.30,...,1.148587,1.621600,0.021340,-0.009370,-0.973554,-0.366987,0.176838,-0.801539,-0.595250,-0.213623
2,india live abroad met guy france party want date,e thapar university thapar university institut...,0,26,20,104,119,4,46,0.09,...,0.375939,-0.390517,0.424014,-0.255889,0.658682,-0.147168,0.167501,1.063089,0.288835,0.669018
3,many people u hate southern state,boyfriend doesnt feel guilty hurt cried tellin...,0,13,30,58,145,1,43,0.02,...,1.141212,0.889100,-0.096269,-0.231384,-0.158132,-0.697380,0.580490,0.345119,0.426423,0.144640
4,consequence bhopal gas tragedy,reason behind bhopal gas tragedy,0,5,9,34,49,3,14,0.21,...,0.536973,-0.854142,0.512441,-0.083297,0.247695,-0.853796,0.882856,0.418120,0.636983,0.418924
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79995,tip making job interview process strategic hot...,tip making job interview process starwood hote...,0,16,16,94,94,15,32,0.47,...,0.063042,-0.795720,0.817972,-0.290288,0.940482,-0.879939,-0.833946,-0.333243,-0.821730,0.785109
79996,best jean band india,best indonesian band,0,8,6,39,32,2,14,0.14,...,-0.517689,-0.199509,0.526762,1.571541,-0.285277,0.841175,0.542463,-1.062503,-0.953382,-1.120422
79997,world like 1880,earth climate 1880,0,7,7,29,35,3,14,0.21,...,-0.817451,-0.812598,1.323160,-0.751704,-0.653077,-0.275874,1.284782,0.375073,1.484241,1.706400
79998,people want become rich term money business,people want become rich,1,13,7,67,35,6,20,0.30,...,1.457030,2.195363,0.214798,-0.978197,-0.962904,0.350843,-1.032875,-0.347113,-1.373212,0.095913


In [31]:
df1 = df1.drop(['question1_vector', 'question2_vector'], axis=1)

In [32]:
df1.columns

Index(['question1', 'question2', 'is_duplicate', 'q1_length', 'q2_length',
       'q1_chars', 'q2_chars', 'word_common', 'total_words', 'word_share',
       ...
       'q2_vec_90', 'q2_vec_91', 'q2_vec_92', 'q2_vec_93', 'q2_vec_94',
       'q2_vec_95', 'q2_vec_96', 'q2_vec_97', 'q2_vec_98', 'q2_vec_99'],
      dtype='object', length=210)

In [58]:
# 7 additional features you want to include
additional_features = ['q1_length', 'q2_length', 'q1_chars', 'q2_chars',
                       'word_common', 'total_words','word_share']

# Select all vector columns (q1_vec_* and q2_vec_*)
q1_vector_columns = [col for col in df1.columns if col.startswith('q1_vec_')]
q2_vector_columns = [col for col in df1.columns if col.startswith('q2_vec_')]

# Combine vector columns and additional features
selected_columns = q1_vector_columns + q2_vector_columns + additional_features

# Create the feature set X
X = df1[selected_columns]
y=df1['is_duplicate']


In [59]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=42)

In [60]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred=rf.predict(X_test)

In [61]:
from sklearn.metrics import accuracy_score,confusion_matrix
print(accuracy_score(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

0.7655833333333333
[[13243  1954]
 [ 3672  5131]]


In [37]:
# rf_params={"max_depth":[8,10,15,17,None],
#            "max_features":[5,7,"auto",8],
#             "min_samples_split": [2, 8, 15, 20],
#             "n_estimators": [100, 200, 500, 1000]}
# randomcv_models = [("RF", RandomForestClassifier(), rf_params)]
# from sklearn.model_selection import RandomizedSearchCV
# model_param={}
# for name,model,params in randomcv_models:
#   random=RandomizedSearchCV(estimator=model,
#                             param_distributions=params,
#                             cv=3,
#                             verbose=2,
#                             n_jobs=-1)
#   random.fit(X_train,y_train)
#   model_param[name]=random.best_params_
# for model_name in model_param:
#   print(f"---------------- Best Params for {model_name} -------------------")
#   print(model_param[model_name])

Fitting 3 folds for each of 10 candidates, totalling 30 fits
---------------- Best Params for RF -------------------
{'n_estimators': 100, 'min_samples_split': 2, 'max_features': 'auto', 'max_depth': None}


In [42]:
# rf_tunned=RandomForestClassifier(n_estimators= 100, min_samples_split= 2, max_features='auto', max_depth=None)

In [44]:
# rf_tunned.fit(X_train,y_train)
# y_pred=rf_tunned.predict(X_test)
# print(accuracy_score(y_test,y_pred))
# print(confusion_matrix(y_test,y_pred))

0.759375
[[13559  1638]
 [ 4137  4666]]


In [62]:
def count_common_words(string1, string2):
    # Split strings into words, convert to lowercase, and create sets
    set1 = set(word.lower().strip() for word in string1.split())
    set2 = set(word.lower().strip() for word in string2.split())
    
    # Find the intersection of the two sets
    common_words = set1 & set2
    
    # Return the count of common words
    return len(common_words)


In [63]:
def transform(q1, q2):
    input_query = []
    
    ## Preprocess
    q1 = text_cleaning(q1)
    q2 = text_cleaning(q2)
    
    ## Fetching excess column features:
    input_query.append(fetch_length(q1))
    input_query.append(fetch_length(q2))
    input_query.append(len(q1))
    input_query.append(len(q2))
    input_query.append(count_common_words(q1, q2))
    input_query.append(fetch_length(q1) + fetch_length(q2))
    input_query.append(count_common_words(q1, q2) / (fetch_length(q1) + fetch_length(q2)))
    
    # Removing stop words from strings:
    q1_new = remove_stopwords(q1)
    q2_new = remove_stopwords(q2)
    
    # Changing into vectors:
    q1_vector = sentence_to_vector(q1_new, word2vec_model, 100)
    q2_vector = sentence_to_vector(q2_new, word2vec_model, 100)
    
    # Combine the features and vectors using np.hstack
    return np.hstack((q1_vector, q2_vector, np.array(input_query)))


In [64]:
q1 = 'Where is the capital of India?'
q2 = 'What is the current capital of Pakistan?'
q3 = 'Which city serves as the capital of India?'
q4 = 'What is the business capital of India?'

In [70]:
rf.predict(transform(q4, q2).reshape(1, -1))

array([0], dtype=int64)

In [71]:
word2vec_model

In [72]:
import pickle

pickle.dump(rf,open('model.pkl','wb'))
pickle.dump(word2vec_model,open('word2vec_model.pkl','wb'))